In [ ]:
# Import-Module 'classExplorer', 'ninmonkey.console' *>$null -pass
# Import-Module 'classExplorer', 'ninmonkey.console' *>$null -pass
[byte[]]$bytes = [Convert]::FromHexString('ff' * 8 -join '') # euivalent: ('ffffffff' + 'ffffffff') #

h1 'signed, all low'
[int32]$a = 500
[int16]$b = 500
[int16]$c = 500
[Guid]::new($a, $b, $c, $bytes)

h1 'unsigned, all low'
[uint32]$ua = 500
[uint16]$ub = 500
[uint16]$uc = 500
[Guid]::new($ua, $ub, $uc, $bytes)


h1 'unsigned, max'  # Throws
[uint32]$ua = [uint32]::maxValue
[uint16]$ub = [uint16]::maxValue
[uint16]$uc = [uint16]::maxValue
[Guid]::new( $ua, $ub, $uc, $bytes)
h1 'extra explicit'
[Guid]::new( [uint32]$ua, [uint16]$ub, [uint16]$uc, $bytes)


# signed, all low


Guid
----
000001f4-01f4-01f4-ffff-ffffffffffff

# unsigned, all low

000001f4-01f4-01f4-ffff-ffffffffffff

# unsigned, max

MethodException: 
Line |
  22 |  [Guid]::new( $ua, $ub, $uc, $bytes)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot convert argument "a", with value: "4294967295", for ".ctor" to type "System.Int32": "Cannot convert value "4294967295" to type "System.Int32". Error: "Value was either too large or too small for an Int32.""

# extra explicit

MethodException: 
Line |
  24 |  [Guid]::new( [uint32]$ua, [uint16]$ub, [uint16]$uc, $bytes)
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot convert argument "a", with value: "4294967295", for ".ctor" to type "System.Int32": "Cannot convert value "4294967295" to type "System.Int32". Error: "Value was either too large or too small for an Int32.""

